<a href="https://colab.research.google.com/github/ednavivianasegura/ERAP_CursoPython/blob/main/Modulo3_RedesNeuronales/RedesNeuronales_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aplicacion de las redes neuronales en problemas de regresión

In [ ]:
# Ejemplo de regresión con dataset no lineal (Friedman #1)

from sklearn.datasets import make_friedman1
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import os

In [ ]:
# Clonar repositorio
!git clone https://github.com/ednavivianasegura/ERAP_CursoPython/
os.chdir("/content/ERAP_CursoPython/Modulo2_Fundamentos_AI")

fatal: destination path 'ERAP_CursoPython' already exists and is not an empty directory.


## El conjunto de datos sintético Friedman #1,

un clásico para probar métodos de regresión no lineales. Este problema es famoso porque:

La relación entre $X$ e  $y$  es altamente no lineal
La regresión lineal falla al capturar esos efectos, mientras que una red neuronal con capas ocultas sí puede aproximarlos.

$$
y=10sin(π X_1 X_2)+20(X_3−0.5)^2 +10X_4+5X_5 +ε
$$

In [ ]:
# 1. Crear datos
X, y = make_friedman1(n_samples=1000, n_features=10, noise=1.0, random_state=42)

In [ ]:
# 2. Dividir y escalar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

In [ ]:
# 3. Modelo base: regresión lineal
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)
print("Regresión Lineal - MSE:", mean_squared_error(y_test, y_pred_lr))
print("Regresión Lineal - R2:", r2_score(y_test, y_pred_lr))

In [ ]:
# 4. Red neuronal con Keras


# compilar


# Entrenamiento

# Evaluación


In [ ]:
# 5. Comparación gráfica
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.scatter(y_test, y_pred_lr, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--')
plt.title("Lineal: Predicción vs Real")
plt.xlabel("y real")
plt.ylabel("y predicho")

plt.subplot(1,2,2)
plt.scatter(y_test, y_pred_nn, alpha=0.5, color='orange')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--')
plt.title("NN: Predicción vs Real")
plt.xlabel("y real")
plt.ylabel("y predicho")

plt.tight_layout()
plt.show()

In [ ]:
# 6. Curva de pérdida
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title("Curva de pérdida (MSE)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

### Gráfico de residuos vs valores predichos para la red neuronal

In [ ]:
residuos = y_test - y_pred_nn

plt.figure(figsize=(8,5))
plt.scatter(y_pred_nn, residuos, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Valores Predichos')
plt.ylabel('Residuos (y - ŷ)')
plt.title('Gráfico de residuos vs predicción')
plt.grid(True)
plt.show()

### Gráfico de residuos vs valores predichos para la regresión lineal

In [ ]:
residuos = y_test - y_pred_lr

plt.figure(figsize=(8,5))
plt.scatter(y_pred_lr, residuos, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Valores Predichos')
plt.ylabel('Residuos (y - ŷ)')
plt.title('Gráfico de residuos vs predicción')
plt.grid(True)
plt.show()

# Ahora vamos (nuevamente) a calcular el precio del oro, con base en las otras variables

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
# 1. Cargar datos

df = pd.read_csv('data/GoldUP.csv')
df

,Date,Gold_Price,Crude_Oil,Interest_Rate,USD_INR,Sensex,CPI,USD_Index
0,01-10-2000,4538,1455.51,8.00,46.318297,3711.02,37.23,116.65
1,01-11-2000,4483,1512.47,8.00,46.783613,3997.99,37.31,115.24
2,01-12-2000,4541,1178.11,8.00,46.745856,3972.12,36.98,109.56
3,01-01-2001,4466,1208.18,8.00,46.536033,4326.72,36.90,110.52
4,01-02-2001,4370,1267.18,7.50,46.514595,4247.04,36.73,112.01
...,...,...,...,...,...,...,...,...
234,01-04-2020,45504,1603.02,4.65,76.222334,33717.62,125.85,99.57
235,01-05-2020,46343,2298.55,4.25,75.666063,32424.10,126.24,98.34
236,01-06-2020,47315,2987.46,4.25,75.763427,34915.80,127.00,97.39
237,01-07-2020,49878,3156.01,4.25,74.908993,37606.89,128.53,93.35


In [ ]:
# 2. Preprocesamiento básico
df = df.drop(['Date'], axis=1)
# Separar características y objetivo
y = df['Gold_Price'].values
X = df.drop('Gold_Price', axis=1).values

# División en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

# Escalado de características
dist_scaler = StandardScaler()
X_train_scaled = dist_scaler.fit_transform(X_train)
X_test_scaled = dist_scaler.transform(X_test)

In [ ]:
# 3. Definición del modelo MLP
model = Sequential([
    )  # salida para regresión
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

In [ ]:
# 4. Entrenamiento


In [ ]:
# 5. Evaluación
# Predicciones
y_pred_train = model.predict(X_train_scaled).flatten()
y_pred_test = model.predict(X_test_scaled).flatten()

# Métricas
mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)
print(f"Train MSE: {mse_train:.3f}, R2: {r2_train:.3f}")
print(f"Test  MSE: {mse_test:.3f}, R2: {r2_test:.3f}")

In [ ]:
# 6. Gráficos de análisis
plt.figure(figsize=(12, 5))

# 6.1 Curva de pérdida
plt.subplot(1, 3, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Curva de pérdida')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend()

# 6.2 Predicción vs real (test)
plt.subplot(1, 3, 2)
plt.scatter(y_test, y_pred_test, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--')
plt.title('Predicción vs Real (Test)')
plt.xlabel('Precio real')
plt.ylabel('Precio predicho')

# 6.3 Residuos vs predicción
residuals = y_test - y_pred_test
plt.subplot(1, 3, 3)
plt.scatter(y_pred_test, residuals, alpha=0.6)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuos vs Predicción')
plt.xlabel('Precio predicho')
plt.ylabel('Residuals')

plt.tight_layout()
plt.show()


# Busqueda automática de hiperparámetros

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from tensorflow import keras
from tensorflow.keras import layers

# 1. Cargar y preparar los datos
df = pd.read_csv('data/GoldUP.csv')

# 1-2. Preprocesamiento básico
df = df.drop(['Date'], axis=1)
# Separar características y objetivo
y = df['Gold_Price'].values
X = df.drop('Gold_Price', axis=1).values

# División en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 2. Definir función para crear el modelo
def build_model(n_hidden=1, n_neurons=30, learning_rate=0.01, input_shape=(X_train.shape[1],)):
    model = keras.models.Sequential()
    model.add(keras.Input(shape=input_shape))
    for _ in range(n_hidden):
        model.add(layers.Dense(n_neurons, activation="relu"))
    model.add(layers.Dense(1))  # Salida de regresión
    model.compile(
        loss="mse",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        metrics=["mae"]
    )
    return model

# 3. Definir la grilla de hiperparámetros
param_grid = {
    "n_hidden": [1, 2],
    "n_neurons": [20, 30],
    "learning_rate": [0.01, 0.001],
    "batch_size": [32, 64],
    "epochs": [50]
}

# 4. Búsqueda manual de hiperparámetros
best_rmse = float("inf")
best_params = None
best_model = None

for n_hidden in param_grid["n_hidden"]:
    for n_neurons in param_grid["n_neurons"]:
        for learning_rate in param_grid["learning_rate"]:
            for batch_size in param_grid["batch_size"]:
                print(f"Entrenando modelo con: hidden={n_hidden}, neurons={n_neurons}, lr={learning_rate}, batch={batch_size}")
                model = build_model(n_hidden=n_hidden, n_neurons=n_neurons, learning_rate=learning_rate)
                model.fit(X_train, y_train, epochs=param_grid["epochs"][0], batch_size=batch_size, verbose=0)
                y_pred = model.predict(X_test).flatten()
                rmse = np.sqrt(mean_squared_error(y_test, y_pred))
                r2_test = r2_score(y_test, y_pred)
                print(f"→ RMSE: {rmse:.4f}")
                print(f"r2: {r2_test:.4f}")
                if rmse < best_rmse:
                    best_rmse = rmse
                    best_params = (n_hidden, n_neurons, learning_rate, batch_size)
                    best_model = model

print("\nMejores hiperparámetros encontrados:")
print(f"Hidden layers: {best_params[0]}, Neurons: {best_params[1]}, LR: {best_params[2]}, Batch size: {best_params[3]}")
print(f"Mejor RMSE: {best_rmse:.4f}")
print(f"Final r2: {r2_test:.4f}")

In [ ]:
!pip install --upgrade scikit-learn scikeras

In [ ]:
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Cargar datos
data = fetch_california_housing()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Modelo
def build_model(n_hidden=1, n_neurons=30, learning_rate=0.01):
    model = keras.Sequential()
    model.add(keras.Input(shape=(X.shape[1],)))
    for _ in range(n_hidden):
        model.add(layers.Dense(n_neurons, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(
        loss='mean_squared_error',
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        metrics=['mean_absolute_error']
    )
    return model

# Envolver en un estimador compatible con sklearn
keras_reg = KerasRegressor(model=build_model, verbose=0)

# Definir grilla
param_grid = {
    "model__n_hidden": [1, 2],
    "model__n_neurons": [20, 30],
    "model__learning_rate": [0.01, 0.001],
    "batch_size": [32, 64],
    "epochs": [50]
}

# Grid Search
grid_search = GridSearchCV(keras_reg, param_grid, cv=3, scoring="neg_mean_squared_error")
grid_search.fit(X_train, y_train)

# Evaluación
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Mejores hiperparámetros: {grid_search.best_params_}")
print(f"RMSE en test: {rmse:.4f}")
